In [31]:
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import f1_score
import json

In [32]:
#X = pd.read_csv('./IU_Xray/clean_data.csv')

#Y = pd.read_csv('./IU_Xray/labeled_reports.csv')

#Y['report_id'] = X['uid'].apply(lambda x: 'CXR'+str(x))

#Y = Y[['report_id','Enlarged Cardiomediastinum', 'Cardiomegaly',
#       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
#       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
#       'Fracture', 'Support Devices', 'No Finding']]#.to_csv('chexbert_labels.csv')

#Y = Y.fillna(0)
#Y.replace(-1,1)
#Y.to_csv('./IU_Xray/chexbert_labels.csv')

In [33]:
labels_path = '../../IU_data/chexbert_labeling/'

In [34]:
X = pd.read_csv(labels_path+'report_impressions.csv')

Y = pd.read_csv(labels_path+'labeled_reports.csv')

Y['report_id'] = X['Id']

Y = Y[['report_id','Enlarged Cardiomediastinum', 'Cardiomegaly',
      'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
      'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
      'Fracture', 'Support Devices', 'No Finding']]#.to_csv('chexbert_labels.csv')

Y = Y.fillna(0)
Y = Y.replace(-1,1)
Y.to_csv(labels_path+'chexbert_labels.csv')

In [35]:
Y.shape

(3921, 15)

In [53]:
len(Y[Y['Cardiomegaly']==1]['report_id'].tolist())

331

In [54]:
sum(Y.sum(axis=1)==0)

289

In [55]:
Y.sum()

report_id                     CXR3967CXR3188CXR1328CXR2982CXR738CXR171CXR127...
Enlarged Cardiomediastinum                                                   75
Cardiomegaly                                                                331
Lung Opacity                                                                399
Lung Lesion                                                                 132
Edema                                                                       101
Consolidation                                                                37
Pneumonia                                                                   120
Atelectasis                                                                 296
Pneumothorax                                                                 24
Pleural Effusion                                                            164
Pleural Other                                                                46
Fracture                                

In [56]:
path='/media/zaheer/Data/Image_Text_Datasets/IU_Xray/latest/One_Image_norm_abnorm_split/r2gen_annotations/'

In [57]:
with open(path+'images_features_2d_r2gen.pickle','rb') as features:
#with open(path+'densenet_report_features.pkl','rb') as features:
    data=pickle.load(features)

In [58]:
data['CXR1'][0].shape

(1, 7, 7, 1024)

In [59]:
feature_shape = 1024

In [66]:
tags = pd.read_csv(labels_path+'chexbert_labels.csv')
    
pneumonia_reports = tags[tags['Atelectasis']==1]['report_id'].tolist()

In [67]:
len(pneumonia_reports)

296

In [68]:
def clean_r2gen_annotations():
    
    tags = pd.read_csv(labels_path+'chexbert_labels.csv')
    
    reports = tags['report_id'].tolist()
    
    #print(reports)
    
    with open(path+'new_annotation.json', 'rb') as f:
        full_records = json.load(f)

    splits=['train','val','test']
    
    new_images_records={}
    
    for s in splits:
        new_records=[]
        records = full_records[s]
        for r in records:
            if r['image_path'][0].split('_')[0] in pneumonia_reports: 
                new_records.append(r)
        print(len(new_records))        
        new_images_records[s]=new_records
        
    with open(path+'Atelectasis_annotation.json', 'w') as f:
        json.dump(new_images_records , f, ensure_ascii = False )
        
    return new_images_records

annot = clean_r2gen_annotations()

def fetch_tags(image_id, tags):

    new_tags=[]
    image_ids =[]

    for image in image_id:
        
        
        tag=(tags[tags.report_id == image.split('_')[0]]['Pneumonia'])
        if tag.shape[0] != 0:
            tag = tag.values.tolist()[0]
            image_ids.append(image)
            new_tags.append(tag)

    return image_ids, new_tags


def convert_r2gen_annotations():
    with open(path+'pneumonia_annotation.json', 'rb') as f:
        full_records = json.load(f)

    splits = ['train','val','test']
    
    new_images_records = {}
    
    for s in splits:
        images=[]
        records = full_records[s]
        for r in records:
            for image in r['image_path']:
                images.append(image)
                
        new_images_records[s]=images
    return new_images_records

convert_r2gen_annotations()

def load_preprocess_data():
    tags = pd.read_csv(labels_path+'chexbert_labels.csv')
    converted_records = convert_r2gen_annotations()
    x_train = converted_records['train']
    x_test = converted_records['test']
    x_val = converted_records['val']
    #print(train.shape, train.columns)

    x_train, train_tags=fetch_tags(x_train,tags)
    #print(train_tags)
    #y_train = mlb.fit_transform([train_tags])
    y_train=np.array(train_tags)
    #print(y_train)

    #print(y_train.head())
    
    
    x_test, test_tags=fetch_tags(x_test,tags)
    y_test = np.array(test_tags)
    
    
    x_val, val_tags=fetch_tags(x_val,tags)
    y_val = np.array(val_tags)
    return x_train,y_train,x_test,y_test, x_val,y_val



143
18
31


In [12]:
def load_features(split):
    features = []#np.empty((0,7,7,1024))
    
    for image in split:
        #print(image)
        report_id = image.split('_')[0]
        image_id = image.split('.')[0].split('/')[1]
        features.append(data[report_id][int(image_id)])
    features=np.array(features)
    features=np.reshape(features,(-1,49,feature_shape))
    
    return features

In [13]:
def class_model(n):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(49,feature_shape)))
    
    model.add(layers.GlobalAveragePooling1D())
    #model.add(layers.Dense(128,activation='relu'))
    #model.add(layers.LeakyReLU(alpha=0.05))
    #model.add(layers.Dense(64,activation='relu'))
    #model.add(layers.Dropout(0.4))
    model.add(layers.Dense(n, activation="sigmoid"))

    model.summary()
    
    return model



In [14]:
images_train,y_train,images_test,y_test, images_val,y_val=load_preprocess_data()

In [15]:

train_features=load_features(images_train)
test_features=load_features(images_test)
val_features=load_features(images_val)

In [16]:
y_train.shape

(4126,)

In [17]:
model=class_model(1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling1d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 1,025
Trainable params: 1,025
Non-trainable params: 0
_________________________________________________________________


In [18]:
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
])

model.fit(train_features, y_train, epochs=100, callbacks=[earlyStopping],validation_data=(val_features, y_val), batch_size=16)

test_tags=model.predict(test_features)
val_tags=model.predict(val_features)

Train on 4126 samples, validate on 592 samples
Epoch 1/100
4126/4126 [==============================] - 1s 363us/sample - loss: 0.0854 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: -0.0398 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
4126/4126 [==============================] - 1s 180us/sample - loss: 0.0309 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: -0.0554 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/100
4126/4126 [==============================] - 1s 178us/sample - loss: 0.0271 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: -0.0601 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/100
4126/4126 [==============================] - 1s 184us/sample - loss: 0.0237 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: -0.0678 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/100
4126/4126 [==============================] - 1s 184us/sample - loss: 0.0220 - precision: 0.0000e+00 - recall: 0.0000e

KeyboardInterrupt: 

In [ ]:
test_tags=np.where(test_tags<0.50,0,1)
test_tags.shape

In [ ]:
val_tags=np.where(val_tags<0.50,0,1)
#val_tags

In [ ]:
y_test_0=np.where(y_test==-1.0,0,y_test)

In [ ]:
y_test_1=np.where(y_test==-1.0,1,y_test)

In [ ]:
#val_tags=[i[0] for i in val_tags.tolist()]
#val_tags=pd.Series(val_tags)

In [ ]:
#print(f1_score(y_test,test_tags))
print(y_test.shape, test_tags.shape)

In [ ]:
#np.max(y_test_1[:,cidx])

In [ ]:
print(f1_score(pd.Series(y_test_1), pd.Series(test_tags[:,0])))

In [ ]:
for cidx in range(y_test.shape[1]):
    print(f1_score(pd.Series(y_test_0[:,cidx]), pd.Series(test_tags[:,cidx])))

In [ ]:
m=tf.keras.metrics.Recall()
m.update_state(y_test,test_tags)
m.result().numpy()

In [ ]:
import pickle

all_tags={}

test_true = np.array(y_test)
for idx,image in enumerate(images_test):
    #all_tags[image.split('_')[0]]=test_tags[idx,:]
    all_tags[image.split('_')[0]]=test_true[idx,:]

val_true = np.array(y_val)
for idx,image in enumerate(images_val):
    #all_tags[image.split('_')[0]]=val_tags[idx,:]
    all_tags[image.split('_')[0]]=val_true[idx,:]
    
train_true=np.array(y_train)
for idx,image in enumerate(images_train):
    all_tags[image.split('_')[0]]=train_true[idx,:]


#print(all_tags['CXR3614'])
#with open(path+'chexbert_true_chex_r2gen.pkl','wb') as file:
#with open(path+'chexbert_dense_r2gen.pkl','wb') as file:
    #pickle.dump(all_tags, file, protocol=2)
    